In [ ]:
#| default_exp netana_L07

In [ ]:
from graph_tool import Graph
#| hide
from nbdev.showdoc import *

# Lecture 07
> Communities, Clusters, Motifs

---
* Community finding
* Modularity
* Clustering


In [ ]:
import numpy as np
from graph_tool import Graph
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt

In [ ]:

# L is Laplacian Matrix
# D is Diagonal Degree Matrix




So basically in the last lecture we talked about network partitioning which separated a network into $K$ equal size parts while minimizing the cut size.

The question that is now obvious is what happens if we don't want balanced partitions because it wouldn't represent the structure in our network well.

Partitioning forces a structure upon our network that might or might not be there essentially. But we would rather want to find out what the internal grouping structure of a network, without making assumptions about it beforehand.

This is where **community detection** comes in.


## Community
> This section tells us how to qualify what a community is, how it differs from other groupings like partitions or node roles, introduces different types of detection methods, discusses community detection as a hard problem, and gives an example of how challenging and relevant this is at scale.

A community is a group of nodes that is formed by mutual adjacency, proximity, or reachability. It suggests the presence of real-world interaction for a common purpose.

Nodes in a community are more likely to connect to each other than to nodes from other communities.

We can capture this by: Given connected subgraph $C$, let again the internal degree $d_i^{int}$ of node $i$ be the number of links to other nodes in $C$, external degree $k_i^{ext}$ number of links to the rest.

Then $C$ is a strong community if each node in $C$ has $k_i^{int}(C) > k_i^{ext}(C)$

$C$ is a weak community if $\sum_{i\in C} k_i^{int}(C) > \sum_{i\in C}k_i^{ext}(C)$

E.g. each clique is a strong community

Note the difference to

* Partition - we do not assume predefined numbers or sizes of groups, or non-overlap
* Nodes with similar roles - specific characteristics or function of single nodes in a network (e.g. peripheral, center of a star, member of a clique). These nodes belong to a category but don't necessarily form a community.

An equivalence relation of the vertices corresponds to a partitioning (into equivalent classes) and a role assignment, not communities.

Example: Network from data of belgian mobile phone company. 2.6 million users, calls over 6 months. Blondel et al.'s analysis reveals 261 communities of >100 customers (node size proportional to #customers, color language spoken, red French, green Dutch



Our goal here will be to find algorithms that can detect the presence and the size and type of each of these communities in our network.



## Community Detection
> How to Identify Communities?

More on this topic in Chapter 14 Newman

Beyond the term Community there is also a mayor problem of detecting communities in networks. This is not easy and not every network does necessarily have a community. So algorithms have to be able to detect them if they are there but also be able to say that the arent. It seems like there are two mayor methods. They are handeled in two subsections, namely Modularity and Divisive, which seem to be just two different methods for doing this.


* Community detection: Major research topic
* Many definitions of criteria and many methods exist.

Two main aspects: An objective and a method to optimise it.

Paradigms to build communities:

* Agglomerative: start from single entities, increase
* Divisive: Start with full graph, remove edges, e.g. based on the importance of edges.

Sometimes we have additional constraints, e.g. the number of groups.

Many goals constitute hard optimization problems, and thus many heuristics and approximation methods exist.

Note: There simply might not be good communities in a graph (Question: How do we know?).

What about the number of possible solutions here?

Given by Bell number, recursively $B_{N+1} = \sum_{k=0}^N B_k, B_0 = 1$

Dobinski formula $B_{N+1} = \frac{1}{e}\sum_{k=0}^\inf \frac{k^N}{k!}$

1, 1, 2, 5, 15, 52, 203, 877, 4140, 21147, 115975, 678570, 4213597, 27644437,...


### Community Detection Measures — Modularity

Modularity: An objective for community detection. Measures the extent to which like is connected to like (within a community) for a given partitioning in communities.

Strictly less than one, positive if there are more "like-to-like" edges than expected by random chance (configuration model). Negative values $(\geq-0.5)$ indicate disassortative mixing.
$$Q = \frac{1}{\underbrace{2M}_{\text{Normalise}}}\sum_{ij}\left(\underbrace{A_{ij}}_{\text{Actual eges}}-\frac{k_ik_j}{\underbrace{2M}_{\text{Expected edges}}}\right)\underbrace{\delta_{g_ig_j}}_{\text{Given partitioning\ in communities }g_k}$$

Note that $\sum_{\text{edges}(i,j)}\delta_{g_ig_j} = \frac{1}{2}\sum g_{ij}A_{ij} \delta_{g_ig_j}$ is the number of edges between nodes in the same community.

Expected connection in a random null model with degree distribution $\frac{k_i k_j}{2M}$ as we have $k_i$ opportunities to end an edge from i that one of the stubs at j, which have a fraction of $\frac{k_j}{2M}$ of all stubs.

Sometimes the equivalent community-only formula is used $$Q = \frac{1}{2M}\sum_c \left( E_c - \gamma \frac{K_c^2}{2M}\right)$$

with $E_c$ edges in community $c, K_c$ sum of degrees of nodes in $c$, and $\gamma$ an additional resolution parameter.
Communities should have at least density $\gamma$ while between them, it should be lower. Higher $\gamma$ leads to more communities. See e.g. networkx implementation.

(Note that we can use this formula for easier optimisation / computation - merge edge lists.)

* Maximum modularity seems to work well for communities.
* It can be used to evaluate quality of results from community detection algorithms or as objective for an algorithm.
* Examples for Optimal Partition with high $M$, suboptimal partition with low $M$, single community with $M=0$, negative modularity with negative $M$.
* Maximum modularity is not always intuitive, examples with non-local behaviour, a clique $K_3$ with leaves, scaling behaviour, clusters represented as colours.
* Note: Similarity to assortativity, also finds communities in null model networks (model ignores search for communities in statistical manner - overfitting, finds high modularity communities, p-hacking)
* Resolution limit max. $\sqrt{2E}$ communities: Prefers communities above a certain size (underfitting).


### Modularity Methods

#### Community Detection — Agglomerative

Maximum modularity problem is unfortunately NP-complete.
Simple heuristic: Greedy joining

Iteratively joins nodes if the move increases the new partition's modularity ($N-1$ merges, $\mathcal{O}(N^2 log N)$ — similar to our KL partitioning, up to engineering).

* Step 1: Assign each node a community of its own. Hence we start with $N$ communities.
* Step 2: Inspect each pair of communities connected by at least one link and compute the modularity variation obtained if we merge these two communities.
* Step 3: Identify the community paris for which $\Delta M$ is the largest and merge them. Note that modularity of a particular partition is always calculated from the full topology of the network.
* Step 4: Repeat step 2 until all nodes are merged into a single community.
* Step 5: Record for each step and select the partition for which the modularity is maximal.

Unbounded approximation ratio.

Compromise of very fast and reasonable quality: Louvrain method (Bondel et al.) Agglomerative multilevel method.

Consider individual node moves to different communities until no improvement is possible (Consider neighbours community). After initial merging, create new network with communities as nodes and repeat. Depending on structure reported to be somewhere between $\mathcal{O}(N \log N)$ and $\mathcal{O}(N^2)$ in practice.

Problem: Communities don't need to be connected $\rightarrow$ Improvement Leiden algorithm: Refinement Step

**Modularity Take-away**

* Concept and formula
* Heuristics

Proceed with caution - it looks simple and interpretable with reasonable null model comparison, but results might be far from the expected.
However, you will find a lot of methods in many tools, thus our discussion of it.


#### Community Detection — Divisive

Create communities by iteratively removing edges that connect nodes with low similarity.

Creates a hierarchy.

E.g. Girvan-Newman:

* Step 1: Define a centrality measure for links
    * Link betweenness is the number of shortest paths between all node paris that run along a link (relative to all). $\mathcal{O}(MN)$ or for sparse  $\mathcal{O}(N^2)$
    * Random-walk betweenness. A pair of nodes $m$ and $n$ are chosen at random. A walker starts at $m$, following each adjacent link with equal probability unitl it reaches $n$. Random walk betweenness $x_{ij}$ is the probability that the link $i \rightarrow j$ was crossed by the walker after averaging over all possible choices for starting nodes $m$ and $n$ $\mathcal{O}(MN^2)$
* Step 2: Hierarchical Clustering
    * Compute the centrality of each link
    * Remove the link with the largest centrality; in case of a tie, choose one randomly (!bad).
    * Recalculate the centrality of each link for the altered network.
    * Repeat until all links are removed (yields a dendogram).


Computational complexity:

* Step 1a (calculation betweenness centrality): $\mathcal{O}(N^2)$ for sparse networks
* Step 1b (Recalculation of betweenness $\mathcal{O}(N^3)$ centrality for all links: $\mathcal{O}(LN^2)$



#### How does the Divisive Method for community detection work?

* What is the process of finding communities in network data?

* What is modularity? How is it defined?

In [ ]:
 # What kind of code can we write about community detection? We can probably load any graph and check if there is a community present in it and then get the characteristic metrics for these communities that we can then report on.

def find_community(graph: Graph):
    return {}

def report_communities(communities):
    pass

In [ ]:

G = Graph()

community = find_community(G) # so something like this could be interesting.
report_communities(community) # And then using this community data structure we could maybe report on it.
# Now i definately don't know if this works this way, at least a good guess is that something exists here that we can report on and that we can acutally do in code and probably also usign functions that are already implemented either in graph_tool or networkx.



## Clustering

* Clustering groups data points from a data set into subsets (clusters) that are homogeneous (compared to the rest - adding points might change grouping!)
* In data science/machine learning is often considered unsupervised learning
* Require some notion of similarity
* Usually similarity matrix $S$ (or dissimilarity/distance)
1) Define some distance / similarity
2) Apply an optimisation that optimises an objective function, e.g. "close" points in clusters (e.g. k-means)

* Graph clustering partitions the nodes (usually) of a graph into clusters, i.e. node sets. Clusters might be disjoint, overlapping, or nested.
* Often: goal of having many intra-cluster edges and few inter-cluster edges. The most popular approach for graph clustering is extraction of tightly connected subgroups of nodes (which we termed community detection)
* Many graph clustering techniques using this approach are based on agglomerative clustering algorithms.
* Often clustering in general form is less focused on structure, rather attributes (cluster might be disconnected in the network model — but you might argue there is another network on the attributes) and without the assumption that clusters are (more) densely connected and sparse connectivity in between.
* In practice also clustering of multivariate data / attributed graphs where topological structure is only a part.
* Examples: Clustering based on density of connections vs. clustering based on commonality of neighbors

Relation Clustering vs. Community Detection

* Several sources: Clustering $\approx$ Community detection
* Several sources: Clustering $\neq$ Community detection
* Several sources: Clustering $\subset$ Community detection (e.g. Newman)
* Several sources: Clustering $\supset$ Community detection (e.g. Bader et al.)

Surely strong overlap and roots/focus in, ah, different communities.
E.g. Edge betweenness clustering - Girvan-Newman
Note: We already encountered a reference to clustering, the clustering coeficient.


### Hierarchical Clustering

Hierarchical clustering does not create simply a partitioning, but a multilevel tree structure of partitions, where each cluster except for the root is a subcluster of a supercluster.

* Root cluster (contains all points)
* Levels cut through the hierarchy and constitute clusterings (not necessarily horizontal)
* At the bottom data elements / nodes

From bottom to top the cluster distance can be graphed in a dendogram.

**Clustered graph**
$$C = \left(\underbrace{G}_{\text{Graph}}, \underbrace{T}_{\text{Inclusion Hierarchy}}\right)$$

Clustered drawing: Disjoint regions for disjoint clusters, representing the hierarchy.
Note: In such an inclusion hierarchy, we lose the information on the cluster distance.

* Again, we could proceed in agglomerative or divisive fashion.
* The similarities can be given as input or computed based on input data / structure
* Many methods / classes of methods existing

Similarity:

1) Structural equivalence — share neighbours
2) Regular equivalence — have neighbors that are similar



#### Structural Similarity
Measures:

* Common neighbors $n_ij = \sum_k A_{ik} A_{kj}$ - this disregards the nodes' degrees
* Jaccard coefficient - normalize $n_{ij}$ betwen 0 and 1 by dividing by total number of distinct neighbors $J_{ij} = \frac{n_{ij}}{k_i+k_j-n_ij}$ (count common neighbors once)
* Cosine — defined on vectors:\
    Dot product $x \cdot y = |x||y|\cos\theta \rightarrow \cos \theta = \frac{x\cdot y}{|x||y|}$\
    $0$ if orthogonal / independent\
    Here: Regard rows of adjacency matrix $A$ as vectors and use cosine as similarity Dot product for undirected graphs simply $n_{ij}$ for $i$ and $j$. Thus similarity $$\sigma_{ij} = \cos \theta = \frac{\sum_k A_{ik} A_{kj}}{\sqrt{\sum_k A_{ik}^2}\cdot \sqrt{\sum_k A_{jk}^2}}$$
    Note that for simple unweighted graphs $A_{ik}^l = A_{ik}$ (only 0 or 1), $\sum A_{ik} = k_i$ (degree of i). Thus $$\sigma_{ij} = \frac{n_{ij}}{\sqrt{k_i k_j}}$$

    * Then similarity of i and j is number of common neighbours divided by geometric mean of degrees. (0 by convention for degree zero).
    * Value of 0 for no sharing, 1 for the exact same neighbors.


#### SAHN
Sequential agglomerative hierarchical non-overlapping (SAHN) clustering techniques belong to the classical clustering methods applied heavily in many application domains.

1) Assign each node to its own cluster (N cluster)
2) Iteratively: In each step, decrease number of clusters by one by merging two "most similar" clusters.

Finish either at the desired number of clusters or when only one cluster left.

* Find similar nodes:  (Dis)Similarity / Distance matrix
* $S_{ij}$ contains distance value from node $i$ to node $j$
* Merge similar clusters: Need inter-cluster distance = linkage

Typical linkage strategies (similarity between groups)

* Single linkage - use smallest distance / dissimilarity ("merge by nearest neighbours"). Might create stretches.
* Complete linkage - use largest distance / dissimilarity ("merge by farthest neighbours").
* Average linkage - use average of all pairs of nodes (weighted average after merging)




**Ravasz algorithm**
1) Similarity: connect nodes that share many neighbours\
    Topology overlap matrix $$\sigma_{ij} = \frac{J(i,j)}{min(k_i, k_j) + 1 - \Theta(A_{ij})}$$
$J(i, j)$ simply is $n_{ij}$ plus 1 in case of a direct link (i, j are not common neighbors)
   $$\Theta(A_{ij}) =
   \begin{cases}
        0: A_{ij} \leq 0 \\
        1: \text{else}\\
    \end{cases}$$
   $$\sigma_{ij}=
   \begin{cases}
        0: \text{no direct link or common neighbors}\\
        1: \text{direct link and same neighbors}\\
    \end{cases}$$
2) Merging: what are similarities between groups?
    Average Linkage
3) Building the hierarchy:
    1) Each node is a singleton community
    2) Find the pair with the highest similarity and merge
    3) Calculate similarities for a new community
    4) Repeat from 2) until a single community is left.
4) Build dendrogram, extract organization structure

Running Time:

* Step 1 (calculation similarity matrix): $\mathcal{O}(N^2)$
* Step 2-3 (group similarity): $\mathcal{O}(N^2)$
* Step 4 (dendrogram): $\mathcal{O}(N log N)$
Overall $\mathcal{O}(N^2)$

Example: Topological overlap matrix reordered to show high overlap.


* Similar to the comparison of graphs with the random graph model, we can compare hierarchical structure by a hierarchical random graph model.
* Specify a dendrogram (binary tree) and a set of probabilities at each intersection of the tree.

* The probability for an edge between a pair of nodes is equal to the $p$ stored at the lowest common ancestor.
* Parameters are the probabilities and the tree structure

Assumption in hierarchical clustering: Small modules are nested in larger ones. This is captured by the dendrogram.
But does this faithfully reflect an organisational structure in the network or just and artifact of the approach?
There are obviously networks with such a structure.

#### Hierarchical Netwok Model
1) Start with a fully connected module, e.g. five nodes $K_5$
2) Create four copies and connect "peripheral" nodes to "central" nodes of original, e.g. 25 nodes
3) Repeat from 2): 4 copies and connected peripheral to original central nodes. (125 nodes, ...)

**Scale-free property**
The obtained network is scale-free, its degree distribution following a power-law with $k^\gamma$.
$$\gamma = 1 + \frac{\ln 5}{\ln 4} \simeq 2.16$$

Example:

* Largest hub: Starts with degree 4
* 2nd step: add 4x4
* 3rd step: add 4x4x4
* I.e. $4^n$ per iteration

After iteration n: $k_n(H_i) = \sum_{l=1}^i 4^l$ for hub on level $i$

(there are four copies of the central nodes with degree from last iteration and so on)

__Small $k$ nodes:__

* high clustering coefficient
* their neighbours tend to link to each other in highly interlinked and compact communities.

__High $k$ nodes (hubs):__

* small clustering coefficient
* Connect independent communities.

Green circles: Random rewiring


#### What is Louvrain
Louvrain Community Detection algorithm

#### What is Leiden
Leiden Community Detection algorithm

extracting the community structure of a network based on modularity optimization.

Guarantees that communities are well connected
it is faster
it uncovers better partitions

* What is a cluster? How is it defined?
I think clustering is a whole nother problem again. I could read up on it if there is a similarity or connection between a cluster and a community.



* What is a motif? How is it defined?
I think motif is not part of this lecture maybe the next one, at least i didn't see the word anywhere unitl now.

* I need to get a handle of the formulas, not only to memorize them but to get a pattern recognition working for me, so i can find things again and again. Currently i am only pattern matching. Probably also because the slides dont seem to follow an understandable order. I should rather read the books but i also want to get what he tells us on the slides.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()